In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# 데이터 로드
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full,
    test_size=0.2,
    random_state=42,
    stratify=y_train_full
)



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [2]:
# 정규화
x_train = x_train.astype('float32') / 255.0
x_val   = x_val.astype('float32')   / 255.0
x_test  = x_test.astype('float32')  / 255.0

In [3]:
# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)
datagen.fit(x_train)



In [4]:
# 모델
def build_model():
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(32, 3, padding='same', input_shape=(32,32,3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(32, 3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Dropout(0.25))

    # Block 2
    model.add(layers.Conv2D(64, 3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(64, 3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Dropout(0.25))

    # Block 3
    model.add(layers.Conv2D(128, 3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(128, 3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Dropout(0.25))

    # Fully connected head
    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))

    return model

model = build_model()
model.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 8, 8, 128)      │             

 Total params: 1,345,066 (5.13 MB)

 Trainable params: 1,343,146 (5.12 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [5]:
# 콜백
checkpoint_cb = callbacks.ModelCheckpoint(
    'best_cifar10.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)
earlystop_cb = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)
reduce_lr_cb = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)



In [6]:
# SGD + nag_optimizer
nag_optimizer = optimizers.SGD(
    learning_rate=0.01,
    momentum=0.9,
    nesterov=True
)
model.compile(
    optimizer=nag_optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



In [7]:
# 학습
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb]
)



Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3135 - loss: 2.0046
Epoch 1: val_accuracy improved from -inf to 0.47350, saving model to best_cifar10.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 49ms/step - accuracy: 0.3136 - loss: 2.0042 - val_accuracy: 0.4735 - val_loss: 1.4454 - learning_rate: 0.0100
Epoch 2/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.4723 - loss: 1.4599
Epoch 2: val_accuracy improved from 0.47350 to 0.49500, saving model to best_cifar10.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 48ms/step - accuracy: 0.4724 - loss: 1.4598 - val_accuracy: 0.4950 - val_loss: 1.5859 - learning_rate: 0.0100
Epoch 3/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5448 - loss: 1.2681
Epoch 3: val_accuracy improved from 0.49500 to 0.60780, saving model to best_cifar10.h5


...

625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.7917 - loss: 0.5993 - val_accuracy: 0.8014 - val_loss: 0.5732 - learning_rate: 0.0050
Epoch 23/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.7968 - loss: 0.5813
Epoch 23: val_accuracy did not improve from 0.80140
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.7968 - loss: 0.5813 - val_accuracy: 0.7842 - val_loss: 0.6586 - learning_rate: 0.0050
Epoch 24/100
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.7997 - loss: 0.5846
Epoch 24: val_accuracy improved from 0.80140 to 0.80610, saving model to best_cifar10.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.7997 - loss: 0.5846 - val_accuracy: 0.8061 - val_loss: 0.5651 - learning_rate: 0.0050
Epoch 25/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8095 - loss: 0.5481
Epoch 25: val_accuracy did not improve from 0.80610
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8095 - loss: 0.5481 - val_accuracy: 0.8038 - val_loss: 0.5765 - learning_rate: 0.0050
Epoch 26/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8061 - loss: 0.5538
Epoch 26: val_accuracy improved from 0.80610 to 0.81640, saving model to best_cifar10.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.8061 - loss: 0.5539 - val_accuracy: 0.8164 - val_loss: 0.5449 - learning_rate: 0.0050
Epoch 27/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8075 - loss: 0.5572
Epoch 27: val_accuracy did not improve from 0.81640
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8075 - loss: 0.5572 - val_accuracy: 0.8049 - val_loss: 0.5683 - learning_rate: 0.0050
Epoch 28/100
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8058 - loss: 0.5580
Epoch 28: val_accuracy did not improve from 0.81640
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 45ms/step - accuracy: 0.8058 - loss: 0.5580 - val_accuracy: 0.8152 - val_loss: 0.5393 - learning_rate: 0.0050
Epoch 29/100
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8106 - loss: 0.5391
Epoch 29: val_accuracy improved from 0.81640 to 0.84130, saving model to best_cifar10.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8106 - loss: 0.5391 - val_accuracy: 0.8413 - val_loss: 0.4577 - learning_rate: 0.0050
Epoch 30/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8107 - loss: 0.5478
Epoch 30: val_accuracy did not improve from 0.84130
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8107 - loss: 0.5478 - val_accuracy: 0.8394 - val_loss: 0.4694 - learning_rate: 0.0050
Epoch 31/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8128 - loss: 0.5337
Epoch 31: val_accuracy did not improve from 0.84130
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 48ms/step - accuracy: 0.8128 - loss: 0.5337 - val_accuracy: 0.8195 - val_loss: 0.5249 - learning_rate: 0.0050
Epoch 32/100
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8177 - loss: 0.5227
Epoch 32: val_accuracy did not improve from 0.84130
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8177 - loss: 0.5227 - val_accuracy: 0.7977 - val_loss: 0.6105 - learning_rate: 0.0050
Epoch

625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 44ms/step - accuracy: 0.8144 - loss: 0.5329 - val_accuracy: 0.8491 - val_loss: 0.4404 - learning_rate: 0.0050
Epoch 34/100
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8186 - loss: 0.5259
Epoch 34: val_accuracy did not improve from 0.84910
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.8186 - loss: 0.5259 - val_accuracy: 0.8195 - val_loss: 0.5357 - learning_rate: 0.0050
Epoch 35/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8197 - loss: 0.5157
Epoch 35: val_accuracy did not improve from 0.84910
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.8197 - loss: 0.5157 - val_accuracy: 0.8038 - val_loss: 0.6125 - learning_rate: 0.0050
Epoch 36/100
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8227 - loss: 0.5112
Epoch 36: val_accuracy did not improve from 0.84910
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8227 - loss: 0.5112 - val_accuracy: 0.8276 - val_loss: 0.5067 - learning_rate: 0.0050
Epoch

In [8]:
# 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8378 - loss: 0.4712
Test Loss: 0.4672, Test Accuracy: 0.8389
